<center>

<img src="https://www.itv.org/wp-content/uploads/2021/10/logo-itv.svg" width="500" height="200" />

</center>

# <center> **Especialização em Automação** </center>


## <center> **Inteligência Computacional** </center>

---
### <center> **Redes Neurais** </center>

### <center> Professor: André Almeida Santos</center>

---

# Problema de classificação com dataset de flores Iris

Para este problema de classificação, construiremos uma rede neural artificial totalmente conectada.

**Declaração do problema:**

Antes de abordarmos o problema, vamos entender o que faremos:

- Se alimentarmos nossa rede neural com os dados de Iris, o modelo deverá ser capaz de determinar de que espécie se trata.

**O que precisamos fazer?**

Treinar um modelo de rede neural usando um conjunto de dados conhecido: [Iris flower dataset](https://en.wikipedia.org/wiki/Iris_flower_data_set).

Especificamente, faremos o seguinte:
> - Carregar o conjunto de dados
> - Pré-processar os dados
> - Construir o modelo
> - Definir hiperparâmetros
> - Treinar e avaliar o modelo
> - Salvar e baixar o modelo treinado
> - Realizar previsões com novos dados

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importando o dataset

In [ ]:
# Importando o conjunto de dados do scikit-learn e outros pacotes úteis

from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris

# Vamos definir uma semente aleatória para reprodutibilidade:
seed = 11
np.random.seed(seed)

In [ ]:
# Importando o dataset
iris = load_iris()

In [ ]:
print(iris)

In [ ]:
iris.feature_names

In [ ]:
iris.data

In [ ]:
iris.target

In [ ]:
iris.target_names

Vamos salvar os nomes das classes em uma lista para uso futuro

In [ ]:
classes = iris.target_names

### usando a biblioteca [pandas](https://pandas.pydata.org/) para carregar os dados de uma fonte online no formato csv

In [ ]:
# Usando o pandas para carregar o dataset

iris_csv = pd.read_csv('https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv')

In [ ]:
iris_csv

Podemos usar o iris ou iris_csv como nossa base de dados. Não há diferença no dataset. 

O objetivo de usar o pandas foi demonstrar como carregar dados de fontes csv.

**Daqui em diante usaremos o iris_csv.**

In [ ]:
iris_csv.describe()

## Pré processamento dos dados

A etapa de pré-processamento é muito importante em muitos casos. 

Para este caso, precisaremos apenas fazer uma transformação muito simples: um processo de **one hot encode**, ou, codificação a quente.

In [ ]:
# Definindo os dados de entrada
X = iris_csv.iloc[:, :4]

In [ ]:
X

In [ ]:
# Definindo os dados de saída
y = iris_csv.iloc[:, -1]

In [ ]:
y

In [ ]:
# Alterando as saídas com o módulo LabelBinarizer para criar o One hot encode
from sklearn.preprocessing import LabelBinarizer

# Criando o enconder
enc = LabelBinarizer()
enc.fit(y)

In [ ]:
# Transformando os dados de y
y = enc.transform(y)

In [ ]:
y

In [ ]:
# Separandos os dados em treino e validação
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=seed, stratify=y)

In [ ]:
# Verificando a divisão dos dados

print(f'X_train: {X_train.shape}')
print(f'X_val: {X_val.shape}')
print(f'y_train: {y_train.shape}')
print(f'y_val: {y_val.shape}')

In [ ]:
y_val

## Construindo o modelo e definindo os hiperparâmetos

In [ ]:
# Definindo o input
inputs = keras.Input(shape=(4,), name='entrada')

In [ ]:
# Definindo as camadas intermediárias
out_int = layers.Dense(5, activation="sigmoid", name='int_1')(inputs)

In [ ]:
outputs = layers.Dense(3, name='saida')(out_int)

In [ ]:
modelo = keras.Model(inputs=inputs, outputs=outputs, name="modelo_iris")

In [ ]:
modelo.summary()

In [ ]:
keras.utils.plot_model(modelo, "meu_primeiro_modelo_com_informação_do_shape.png", show_shapes=True)

In [ ]:
modelo.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),
    metrics=[['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]],
)

## Treinando o modelo

In [ ]:
treino = modelo.fit(X_train, y_train, batch_size=4, validation_data=(X_val, y_val), epochs=200)

Plotando os resultados com os dados do histórico de treino

In [ ]:
plt.title('Acurácia do modelo durante treino e validação')
plt.ylabel('Acurácia')
plt.xlabel('Época')
plt.plot(treino.history['accuracy'], label='Treino')
plt.plot(treino.history['val_accuracy'], label='Validação')
plt.legend()
plt.show()

In [ ]:
plt.title('Erro do modelo durante treino e validação')
plt.ylabel('Erro')
plt.xlabel('Época')
plt.plot(treino.history['loss'], label='Treino')
plt.plot(treino.history['val_loss'], label='Validação')
plt.legend()
plt.show()

### Avaliando o modelo treinado

In [ ]:
val_scores = modelo.evaluate(X_val, y_val, verbose=2)

In [ ]:
print("Erro nos dados de validação:", val_scores[0])
print("Acurácia nos dados de validação:", val_scores[1])

## Salvando e carregando o modelo

In [ ]:
modelo.save('modelo_iris')

### Outras formas de salvar o modelo

In [ ]:
# Serializando o modelo com JSON. Aqui é salvo apenas o arquitetura do modelo.
modelo_json = modelo.to_json()
with open("iris_model.json", "w") as json_file:
    json_file.write(modelo_json)

In [ ]:
# Serializar os pesos para o formato HDF5. Aqui é salvo todos os parâmetros do modelo. 
modelo.save_weights("iris_model.h5")
print("Model saved to disk.")

### Carregando

In [ ]:
loaded_model = tf.keras.models.load_model('modelo_iris')

## Realizando novas predições

Vamos criar os novos dados para gerar as predições

In [ ]:
# Dados de entrada

sepal_length = 0
sepal_width = 0
petal_length = 0
petal_width = 0

In [ ]:
entrada = [[sepal_length, sepal_width, petal_length, petal_width]]

In [ ]:
nova_predicao = loaded_model.predict(entrada)

In [ ]:
#Quem é a nova predição???
nova_predicao

Temos que aplicar uma função que identifica o maior valor e a sua posição, pois esse valor representa o **neurônio que foi mais ativado pelo dado de entrada**. 

In [ ]:
valor, idx = tf.math.top_k(nova_predicao)

# Valor recebe o valor da maior probabilidade e idx recebe o inidice do maior valor

In [ ]:
valor

In [ ]:
idx.numpy()

In [ ]:
# Com isso é possível acessar diretamente uma posição na lista classes

classes[idx.numpy()[0][0]]

Formatando uma mensagem após a predição

In [ ]:
# Entada dos dados

sepal_length = 5
sepal_width = 3
petal_length = 5
petal_width = 1.9

entrada = [[sepal_length, sepal_width, petal_length, petal_width]]

# predição
nova_predicao = loaded_model.predict(entrada)

valor, idx = tf.math.top_k(nova_predicao)

# Formatação da mensagem ao usuário
print(f'Tipo de iris predita é: {classes[idx.numpy()[0][0]]}')